In [31]:
%autosave 0

Autosave disabled


The chi^2 test will determine if membership to one group affects membership to another.

In [32]:
import numpy as np
import pandas as pd

from pydataset import data
from scipy import stats

Let's read in the mpg dataset from pydataset!

In [33]:
mpg = data('mpg')
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


Let's do some feature engineering.

Our goal is to compare above/below average mpg to automatic/manual transmission.

In [34]:
mpg['mean_mpg'] = (mpg.cty + mpg.hwy) / 2
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mean_mpg
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0


In [35]:
mpg['mpg_cat'] = pd.qcut(mpg.mean_mpg, 2, labels = ['low_mpg', 'high_mpg'])
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mean_mpg,mpg_cat
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5,high_mpg
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0,high_mpg
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5,high_mpg
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5,high_mpg
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0,high_mpg


In [36]:
mpg.mpg_cat.value_counts()

low_mpg     121
high_mpg    113
Name: mpg_cat, dtype: int64

In [37]:
mpg['trans_bin'] = np.where(mpg.trans.str.startswith('a'), 'auto', 'manual')

Now that we have our categorical features, we can prepare to run a chi^2 contingency test!

First, we need to define our null and alternative hypotheses.

$H_0$ : Transmission type does NOT affect mpg.

$H_a$ : transmission tyoe DOES affect mppg.

We need to generate a crosstab of our two categorical features.

Once we have the crosstab, we can run the test!

In [38]:
ct = pd.crosstab(mpg.mpg_cat, mpg.trans_bin)
ct

trans_bin,auto,manual
mpg_cat,,
low_mpg,95,26
high_mpg,62,51


The test will return four values:
- The test statistic (chi^2)
- The p-value
- The degrees of freedom (sample size minus 1)
- The table of expected values, if the two features were independent of each other

In [40]:
chi , p , degf , exp = stats.chi2_contingency(ct)

Let's evaluate our result using a 95% confidence interval!

In [41]:
a = 0.05
if p < a:
    print('We reject the null hypothesis. There appears to be a relationship.')
else:
    print('We fail to reject the null hypothesis')

We reject the null hypothesis. There appears to be a relationship.


In [42]:
ct

trans_bin,auto,manual
mpg_cat,,
low_mpg,95,26
high_mpg,62,51


In [43]:
exp

array([[81.18376068, 39.81623932],
       [75.81623932, 37.18376068]])

In [44]:
degf

1

In [45]:
chi

13.745374091906687

In [ ]:
pd.c

Bad Chi_2 test below. To many features are compared

In [46]:
bad_ct = pd.crosstab(mpg.trans_bin, mpg['class'])
bad_ct

class,2seater,compact,midsize,minivan,pickup,subcompact,suv
trans_bin,,,,,,,
auto,2,24,29,11,20,16,55
manual,3,23,12,0,13,19,7


In [47]:
chi_two , p_two , degf_two , exp_two =  stats.chi2_contingency(bad_ct)

In [48]:
exp_two

array([[ 3.35470085, 31.53418803, 27.50854701,  7.38034188, 22.14102564,
        23.48290598, 41.5982906 ],
       [ 1.64529915, 15.46581197, 13.49145299,  3.61965812, 10.85897436,
        11.51709402, 20.4017094 ]])

In [49]:
chi_two

33.7700251570052

In [50]:
p_two

7.4513729838250925e-06